In [1]:
import sys
sys.executable

'/Users/arkin/cs229_materials/closed-question-detection/venv/bin/python'

In [27]:
import pandas as pd
import os
FOLDER_PATH="../so_dataset"
from utils import body_strip_tags, just_text, filter_sentence, get_tag_list, remove_filpaths

In [28]:
df_total_features = pd.read_csv(os.path.join(FOLDER_PATH,"so_questions_labelled.csv"))

In [29]:
index = 0
df_total_features['Qid'].iloc[index],df_total_features['Label'].iloc[index]

(64303805, 3)

In [30]:
df_total_features['Title'].iloc[index], df_total_features['Tags'].iloc[index]

('How to convert a Python list of dictionary with same key value into dictionary with list of values',
 '<python><python-3.x><dictionary>')

In [79]:
df_total_features['title'] = df_total_features['Title'].apply(lambda x: ' '.join(filter_sentence(x)).lower())

In [80]:
df_total_features['tag_list'] = df_total_features['Tags'].apply(lambda x: get_tag_list(x))

In [81]:
df_total_features['Body'].iloc[index]

"<p>I have a list of dictionary like this:</p>\n<pre><code>mylist = [\n    {'survival': 124, 'organ': 'Stomach'}\n    {'survival': 42, 'organ': 'Stomach'}\n    ]\n</code></pre>\n<p>I want to change this into a dictionary with same keys to have list of value like so:</p>\n<pre><code>myDict = {\n  &quot;survival&quot;:[124,42],\n  &quot;organ&quot;:[&quot;Stomach&quot;,&quot;Stomach&quot;]\n}\n</code></pre>\n"

In [82]:
body_strip_tags(just_text(df_total_features['Body'].iloc[index]))

'I have a list of dictionary like this:I want to change this into a dictionary with same keys to have list of value like so:'

In [83]:
df_total_features['body'] = df_total_features['Body'].apply(just_text).apply(body_strip_tags)

In [84]:
df_total_features['body'] = df_total_features['body'].apply(remove_filpaths)

In [85]:
from spacy.lang.en import English
import spacy
# nlp = English()
nlp = spacy.load("en_core_web_sm")
tokenizer = nlp.Defaults.create_tokenizer(nlp)

def tokenize(text, remove_stop_words=False, use_lemma=False):
    tokens = []
    doc = tokenizer(text)
    for token in doc:
        if 'd' not in token.shape_ and token.is_alpha:
            if remove_stop_words and token.is_stop:
                continue # ignore stop words
            # print(token.is_stop)
            if use_lemma:
                tokens.append(token.lemma_)
            else:
                tokens.append(token.text)
    return '|'.join(tokens).lower()

In [86]:
df_cleaned = df_total_features.dropna( how='any')

In [88]:
w = df_cleaned['body'].iloc[index]
print(w)
tokenize(w, use_lemma=False, remove_stop_words=False)

I have a list of dictionary like this:I want to change this into a dictionary with same keys to have list of value like so:


'i|have|a|list|of|dictionary|like|this|i|want|to|change|this|into|a|dictionary|with|same|keys|to|have|list|of|value|like|so'

In [89]:
df_cleaned['title'] = df_cleaned['title'].apply(lambda x: tokenize(x, use_lemma=False, remove_stop_words=False))
df_cleaned['body'] = df_cleaned['body'].apply(lambda x: tokenize(x, use_lemma=False, remove_stop_words=False))

In [90]:
df_cleaned.to_csv(os.path.join(FOLDER_PATH, "so_questions_cleaned_rm_stopw.csv"), index=False)

In [91]:
df_cleaned['body']

0       i|have|a|list|of|dictionary|like|this|i|want|t...
1       i|porting|an|elixir|library|to|elm|that|uses|t...
2       trying|to|filter|boolean|field|but|it|brings|t...
3       consider|the|following|code|i|create|a|functio...
4       not|able|to|get|value|of|gridview|cells|i|am|b...
                              ...                        
6143    i|want|to|show|an|section|when|the|checkbox|is...
6144    cpu|use|pytorch|to|train|dnn|but|i|find|that|a...
6145    i|was|install|kubernetes|by|kubespray|my|kubes...
6146    what|is|wrong|in|code|why|the|redirection|to|r...
6147    i|have|a|data|set|with|scores|and|categories|i...
Name: body, Length: 6148, dtype: object

In [92]:
df_cleaned['Label'].value_counts()

0    2000
1    1848
2    1374
3     816
4     110
Name: Label, dtype: int64